### with library

In [23]:
from pydub import AudioSegment

def compress_audio(input_file, output_file, bitrate):
    # Load audio
    audio = AudioSegment.from_file(input_file, format="mp3")
    # Compress audio using the given bitrate
    compressed_audio = audio.export(output_file, format="mp3", bitrate=bitrate)
    return compressed_audio

compress_audio('Wine Bottles Clinking.mp3', "output_audio.mp3", bitrate="10k")


<_io.BufferedRandom name='output_audio.mp3'>

In [30]:
from pydub import AudioSegment
import numpy as np

def compress_audio(input_file, output_file, tolerance):
    # Load audio
    audio = AudioSegment.from_file(input_file, format="mp3")
    # Convert audio to numpy array
    samples = np.array(audio.get_array_of_samples())
    # Transpose array to have samples in rows and channels in columns
    samples = samples.T
    # Compress each channel using SVD
    compressed_samples = np.zeros_like(samples)
    for i in range(2):
        u, s, v = np.linalg.svd(np.atleast_2d(samples[i]).T, full_matrices=False)
        s = np.diag(s)
        subs = int(tolerance*np.shape(s)[0])
        u = u[:,:-subs]
        s = s[:-subs,:-subs]
        v = v[:-subs,:]
        compressed_samples[i] = (u @ s @ v).flatten()
    # Transpose array back to have channels in rows and samples in columns
    compressed_samples = compressed_samples.T
    # Export the compressed audio to a file
    compressed_audio = AudioSegment(
        compressed_samples.tobytes(),
        frame_rate=audio.frame_rate,
        sample_width=audio.sample_width,
        channels=audio.channels
    )
    compressed_audio.export(output_file, format="mp3")

# Example usage:
compress_audio('Wine Bottles Clinking.mp3', "output_audio_SVD.mp3", tolerance=0.1)

